First - data wrangling.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
%matplotlib inline

In [2]:
df = pd.read_csv('train.csv')
df.head()

,Id,Sequence
0,3,"1,3,13,87,1053,28576,2141733,508147108,4021352..."
1,7,"1,2,1,5,5,1,11,16,7,1,23,44,30,9,1,47,112,104,..."
2,8,"1,2,4,5,8,10,16,20,32,40,64,80,128,160,256,320..."
3,11,"1,8,25,83,274,2275,132224,1060067,3312425,1099..."
4,13,"1,111,12211,1343211,147753211,16252853211,1787..."


In [3]:
df.Sequence = df.Sequence.str.split(",")
df.head()

,Id,Sequence
0,3,"[1, 3, 13, 87, 1053, 28576, 2141733, 508147108..."
1,7,"[1, 2, 1, 5, 5, 1, 11, 16, 7, 1, 23, 44, 30, 9..."
2,8,"[1, 2, 4, 5, 8, 10, 16, 20, 32, 40, 64, 80, 12..."
3,11,"[1, 8, 25, 83, 274, 2275, 132224, 1060067, 331..."
4,13,"[1, 111, 12211, 1343211, 147753211, 1625285321..."


In [4]:
def conv(a):
    return list(map(int, a))
df['Sequence'] = df.Sequence.apply(conv)
df.head()

,Id,Sequence
0,3,"[1, 3, 13, 87, 1053, 28576, 2141733, 508147108..."
1,7,"[1, 2, 1, 5, 5, 1, 11, 16, 7, 1, 23, 44, 30, 9..."
2,8,"[1, 2, 4, 5, 8, 10, 16, 20, 32, 40, 64, 80, 12..."
3,11,"[1, 8, 25, 83, 274, 2275, 132224, 1060067, 331..."
4,13,"[1, 111, 12211, 1343211, 147753211, 1625285321..."


Since this is the training dataset, we will use the first n-1 elements of the sequence to predict the next.

In [5]:
def last(a):
    return a[-1]
df['Next'] = df.Sequence.apply(last)
df.head()

,Id,Sequence,Next
0,3,"[1, 3, 13, 87, 1053, 28576, 2141733, 508147108...",11474377948948020660089085281068730
1,7,"[1, 2, 1, 5, 5, 1, 11, 16, 7, 1, 23, 44, 30, 9...",7424
2,8,"[1, 2, 4, 5, 8, 10, 16, 20, 32, 40, 64, 80, 12...",2097152
3,11,"[1, 8, 25, 83, 274, 2275, 132224, 1060067, 331...",18610239435360217
4,13,"[1, 111, 12211, 1343211, 147753211, 1625285321...",28792920887348623853211


In [6]:
def trim(a):
    return a[:-1]
df['Sequence'] = df.Sequence.apply(trim)
df.head()

,Id,Sequence,Next
0,3,"[1, 3, 13, 87, 1053, 28576, 2141733, 508147108...",11474377948948020660089085281068730
1,7,"[1, 2, 1, 5, 5, 1, 11, 16, 7, 1, 23, 44, 30, 9...",7424
2,8,"[1, 2, 4, 5, 8, 10, 16, 20, 32, 40, 64, 80, 12...",2097152
3,11,"[1, 8, 25, 83, 274, 2275, 132224, 1060067, 331...",18610239435360217
4,13,"[1, 111, 12211, 1343211, 147753211, 1625285321...",28792920887348623853211


In [7]:
X = df.Sequence
X.head()

0    [1, 3, 13, 87, 1053, 28576, 2141733, 508147108...
1    [1, 2, 1, 5, 5, 1, 11, 16, 7, 1, 23, 44, 30, 9...
2    [1, 2, 4, 5, 8, 10, 16, 20, 32, 40, 64, 80, 12...
3    [1, 8, 25, 83, 274, 2275, 132224, 1060067, 331...
4    [1, 111, 12211, 1343211, 147753211, 1625285321...
Name: Sequence, dtype: object

In [8]:
y = df.Next
y.head()

0    11474377948948020660089085281068730
1                                   7424
2                                2097152
3                      18610239435360217
4                28792920887348623853211
Name: Next, dtype: object

Data exploration showed that some sequences were only one number long. Thus, removing the last number would result in some empty sequences. These will be removed from the training set.

In [9]:
def notempty(a):
    if len(a) == 0:
        return False
    else:
        return True
empty = df.Sequence.apply(notempty)
df = df[empty]


The following regressions will be based on fitting a model on each sequence individually. 

In [10]:
from sklearn.linear_model import LinearRegression

def linear_predict(a):
    lm = LinearRegression()
    x = np.array(range(0, len(a)))
    lm.fit(x.reshape(x.shape[0], 1), a)
    return int(round(lm.predict(len(a))[0]))
df['Predicted'] = df.Sequence.apply(linear_predict)
df.head()


,Id,Sequence,Next,Predicted
0,3,"[1, 3, 13, 87, 1053, 28576, 2141733, 508147108...",11474377948948020660089085281068730,9686037302534585361341874176
1,7,"[1, 2, 1, 5, 5, 1, 11, 16, 7, 1, 23, 44, 30, 9...",7424,2040
2,8,"[1, 2, 4, 5, 8, 10, 16, 20, 32, 40, 64, 80, 12...",2097152,427497
3,11,"[1, 8, 25, 83, 274, 2275, 132224, 1060067, 331...",18610239435360217,497554387365308
4,13,"[1, 111, 12211, 1343211, 147753211, 1625285321...",28792920887348623853211,95924262224266362880


In [11]:
linear_correct = df[df.Next == df.Predicted]
linear_correct.head()

,Id,Sequence,Next,Predicted
38,77,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, ...",14,14
111,221,"[0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14...",87,87
142,277,"[31, 41, 59, 61, 71, 101, 103, 113, 131, 151, ...",691,691
195,378,"[4, 2, 6, 4, 2, 2, 2, 2, 4, 2, 2, 6, 2, 2, 2, ...",2,2
262,535,"[1, 3, 5, 6, 8, 10, 11, 13, 15, 16, 18, 20, 22...",143,143


In [12]:
# percentage of correct predictions
linear_accuracy = 100 * len(linear_correct)/len(df)
linear_accuracy

4.513391719885101

The linear model wasn't terribly accurate. Would adding more degrees to the polynomial improve accuracy, or would it overfit to the given numbers?

In [13]:
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures


def polynomial_predict(a):
    x = np.array(range(0, len(a)))
    lowest_mse = float("inf")
    global best_p
    for deg in range(1, 6):
        p, res, _, _, _ = np.polyfit(x, a, deg, full = True)
        p = np.poly1d(p)
        if res < lowest_mse:
            lowest_mse = res
            best_p = p
    return int(round(best_p(len(a))))
df['Polynomial Prediction'] = df.Sequence.apply(polynomial_predict)
df.head()

,Id,Sequence,Next,Predicted,Polynomial Prediction
0,3,"[1, 3, 13, 87, 1053, 28576, 2141733, 508147108...",11474377948948020660089085281068730,9686037302534585361341874176,87173233777729598913654030336
1,7,"[1, 2, 1, 5, 5, 1, 11, 16, 7, 1, 23, 44, 30, 9...",7424,2040,-1297
2,8,"[1, 2, 4, 5, 8, 10, 16, 20, 32, 40, 64, 80, 12...",2097152,427497,1675953
3,11,"[1, 8, 25, 83, 274, 2275, 132224, 1060067, 331...",18610239435360217,497554387365308,4396180720475370
4,13,"[1, 111, 12211, 1343211, 147753211, 1625285321...",28792920887348623853211,95924262224266362880,850778089221123342336


In [14]:
polynomial_correct = df[df['Polynomial Prediction'] == df.Next]
polynomial_correct.head()

,Id,Sequence,Next,Predicted,Polynomial Prediction
38,77,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, ...",14,14,14
58,119,"[2, 3, 6, 7, 8, 10, 11, 12, 13, 14, 15, 17, 18]",19,20,19
66,136,"[0, 15, 54, 117, 204, 315, 450, 609, 792, 999,...",18369,15089,18369
80,164,"[2, 5, 10, 11, 15, 18, 19, 21, 25, 26, 28, 35,...",165,163,165
91,181,"[0, 12, 60, 144, 264, 420, 612, 840, 1104, 140...",24420,19974,24420


In [15]:
polynomial_accuracy = 100 * len(polynomial_correct)/len(df)
polynomial_accuracy

6.797319020722248

The polynomial model was more accurate - 6.8% to 4.5%. What types of sequences were better suited for linear models versus polynomial?

In [16]:
lin_beats_poly = df[(df['Polynomial Prediction'] != df.Predicted) & (df.Predicted == df.Next)]
lin_beats_poly.head()

,Id,Sequence,Next,Predicted,Polynomial Prediction
142,277,"[31, 41, 59, 61, 71, 101, 103, 113, 131, 151, ...",691,691,668
262,535,"[1, 3, 5, 6, 8, 10, 11, 13, 15, 16, 18, 20, 22...",143,143,144
288,595,"[1, 1, 1, 1, 0, 1, 1, 0, 2, 1, 1, 2, 0, 1, 1, ...",2,2,3
368,747,"[0, 1, 1, 2, 3, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...",61,61,60
408,831,"[5, 6, 5, 2, 3, 2, 0, 0, 3, 3, 0, 2, 3, 2, 0, ...",0,0,-1


In [17]:
# percentage of linear correct and polynomial incorrect
linear_superiority = 100 * len(lin_beats_poly)/len(df)
linear_superiority

1.8754556874181958

In [18]:
# probability that polynomial incorrect given linear correct
100 * linear_superiority/linear_accuracy

41.55313351498637

In [19]:
poly_beats_lin = df[(df['Polynomial Prediction'] == df.Next) & (df.Predicted != df.Next)]
poly_beats_lin.head()

,Id,Sequence,Next,Predicted,Polynomial Prediction
58,119,"[2, 3, 6, 7, 8, 10, 11, 12, 13, 14, 15, 17, 18]",19,20,19
66,136,"[0, 15, 54, 117, 204, 315, 450, 609, 792, 999,...",18369,15089,18369
80,164,"[2, 5, 10, 11, 15, 18, 19, 21, 25, 26, 28, 35,...",165,163,165
91,181,"[0, 12, 60, 144, 264, 420, 612, 840, 1104, 140...",24420,19974,24420
114,224,"[1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 3, ...",3,4,3


In [20]:
# percentage of polynomial correct and linear incorrect
polynomial_superiority = 100 * len(poly_beats_lin)/len(df)
polynomial_superiority

4.159382988255343

In [21]:
# probability that linear incorrect given polynomial correct
100 * polynomial_superiority/polynomial_accuracy

61.191522357198245

The Polynomial model fared better than the Linear model with respect to overall accuracy. There were cases where the linear model was correct and the polynomial model was incorrect, but far more where the polynomial model was correct and the linear model was incorrect. Thus, further minimization of the mean squared error did not directly translate to a correct prediction.

In [22]:
# percentage of time polynomial model within 10% of correct answer
100 * len(df[(df['Polynomial Prediction'] >= 0.9 * df.Next) & (df['Polynomial Prediction'] <= 1.1 * df.Next)])/len(df)

29.444215075677054

How would a smoothing approach, as typically used with timeseries, affect accuracy? In the following regression, sequences were subdivided, and a regression was fitted onto the means of the individual sections.

In [23]:
def split_list(a, parts=1):
    length = len(a)
    return [a[i*length // parts: (i+1)*length // parts] for i in range(parts)]

def smoothing_linear(a):
    if len(a) < 10:
        parts = len(a)
    else:
        parts = 10
    b = []
    for i in split_list(a, parts=parts):
        b.append(np.array(i).mean())
    lm = LinearRegression()
    x = np.array(range(0, len(b)))
    lm.fit(x.reshape(x.shape[0], 1), b)
    return int(round(lm.predict(len(b))[0]))
df['Smoothing Prediction'] = df.Sequence.apply(smoothing_linear)
df.head()

,Id,Sequence,Next,Predicted,Polynomial Prediction,Smoothing Prediction
0,3,"[1, 3, 13, 87, 1053, 28576, 2141733, 508147108...",11474377948948020660089085281068730,9686037302534585361341874176,87173233777729598913654030336,6295931709284709288162361344
1,7,"[1, 2, 1, 5, 5, 1, 11, 16, 7, 1, 23, 44, 30, 9...",7424,2040,-1297,2063
2,8,"[1, 2, 4, 5, 8, 10, 16, 20, 32, 40, 64, 80, 12...",2097152,427497,1675953,445645
3,11,"[1, 8, 25, 83, 274, 2275, 132224, 1060067, 331...",18610239435360217,497554387365308,4396180720475370,473368953981662
4,13,"[1, 111, 12211, 1343211, 147753211, 1625285321...",28792920887348623853211,95924262224266362880,850778089221123342336,52833944948936867840


In [24]:
smoothing_correct = df[df['Smoothing Prediction'] == df.Next]
smoothing_correct.head()

,Id,Sequence,Next,Predicted,Polynomial Prediction,Smoothing Prediction
58,119,"[2, 3, 6, 7, 8, 10, 11, 12, 13, 14, 15, 17, 18]",19,20,19,19
93,186,"[2, 6, 7, 8, 10, 11, 17, 18, 21, 23, 24, 26, 2...",59,57,55,59
195,378,"[4, 2, 6, 4, 2, 2, 2, 2, 4, 2, 2, 6, 2, 2, 2, ...",2,2,2,2
288,595,"[1, 1, 1, 1, 0, 1, 1, 0, 2, 1, 1, 2, 0, 1, 1, ...",2,2,3,2
293,604,"[1, 2, 1, 1, 1, 1, 1, 1, 2, 0, 2, 1, 1, 1, 2, ...",2,2,2,2


In [25]:
smoothing_accuracy = 100 * len(smoothing_correct)/len(df)
smoothing_accuracy

2.9857957290559476

This method was less accurate than both the linear and polynomial models - using summary statistics is not the best way to increase accuracy.

In [26]:
df[(df['Smoothing Prediction'] == df.Next) & (df.Predicted != df.Next)].head(10)

,Id,Sequence,Next,Predicted,Polynomial Prediction,Smoothing Prediction
58,119,"[2, 3, 6, 7, 8, 10, 11, 12, 13, 14, 15, 17, 18]",19,20,19,19
93,186,"[2, 6, 7, 8, 10, 11, 17, 18, 21, 23, 24, 26, 2...",59,57,55,59
416,850,"[48, 97, 146, 195, 244, 293, 300, 307, 314, 32...",1007,979,1186,1007
495,1003,"[1, 2, 3, 10, 9, 8, 7, 6, 5, 4, 17, 16, 15, 14...",71,69,71,71
977,1930,"[1, 4, 2, 5, 3, 6, 9, 7, 10, 8, 11, 14, 12, 15...",74,72,71,74
1132,2228,"[4, 7, 3, 8, 6, 3, 8, 7, 0, 2, 6, 8, 6, 2, 1, ...",4,5,4,4
1736,3444,"[0, 1, 1, 2, 2, 2, 2, 3, 3, 4, 4, 4, 4, 5, 5, ...",28,27,27,28
1855,3692,"[0, 1, 1, 2, 4, 5, 6, 8, 9, 13, 14, 17, 18, 19...",153,148,155,153
1919,3823,"[1, 2, 1, 4, 2, 1, 7, 14, 7, 16, 8, 4, 2, 1, 5...",149,148,371,149
2049,4092,"[7, 23, 28, 39, 55, 71, 87, 92, 103, 112, 135,...",663,645,640,663


In [27]:
df.head()

,Id,Sequence,Next,Predicted,Polynomial Prediction,Smoothing Prediction
0,3,"[1, 3, 13, 87, 1053, 28576, 2141733, 508147108...",11474377948948020660089085281068730,9686037302534585361341874176,87173233777729598913654030336,6295931709284709288162361344
1,7,"[1, 2, 1, 5, 5, 1, 11, 16, 7, 1, 23, 44, 30, 9...",7424,2040,-1297,2063
2,8,"[1, 2, 4, 5, 8, 10, 16, 20, 32, 40, 64, 80, 12...",2097152,427497,1675953,445645
3,11,"[1, 8, 25, 83, 274, 2275, 132224, 1060067, 331...",18610239435360217,497554387365308,4396180720475370,473368953981662
4,13,"[1, 111, 12211, 1343211, 147753211, 1625285321...",28792920887348623853211,95924262224266362880,850778089221123342336,52833944948936867840


Maybe accuracy isn't the best way to test for the strength of the model. It is unfeasible to predict the future with 100% accuracy, which justifies a switch to a method that rewards being close enough. 

In statistics, the R^2 value is frequently used as a measure of a model's effectiveness. To aid in the calculations of correlation, we define the neglog function - the same function used in data exploration to shrink the massive numbers at play. 

Note that since all values are integers, the sign of each number is preserved - if there were numbers in the range (-1, 0) or (0, 1), their signs would be flipped. 

In [28]:
import math

def neglog(a):
    if a < 0:
        return -1 * math.log(float(a * -1))
    if a == 0:
        return 0
    else:
        return math.log(float(a))

np.corrcoef(df.Next.apply(neglog), df.Predicted.apply(neglog))[0, 1] ** 2

0.55954293433803171

In [29]:
np.corrcoef(df.Next.apply(neglog), df['Polynomial Prediction'].apply(neglog))[0, 1] ** 2

0.50482907164235291

In [30]:
np.corrcoef(df.Next.apply(neglog), df['Smoothing Prediction'].apply(neglog))[0, 1] ** 2

0.56011314113444977

Woah! The R^2 values are inversely proportional to accuracy.  R^2 describes the percentage of variation in predictions that is due to variation in the sequences themselves. Even though smoothing was the worst accuracy-wise, it had the best R^2 score, meaning it was closer overall. It's interesting to note that the polynomial prediction must have been more hit-or-miss: it was more accurate, but when it was off, it was off by more substantial amounts.

However, these methods were all pretty inaccurate. Perhaps instead of fitting models to each sequence individually, it makes more sense to consider them as a whole. 